### Dependency Inversion Principle 

The DIP does not relate to the Dependency Injection 

DIP states that high classes or modules in your code should not directly depend on the low level modules, instead they should depend on **abstractions**. 

Now what do we mean by abstraction here, nothing just some sort of abstract class or class with abstract methods. 

So inshort you should just depend upon interfaces rather than concrete implementations, because that way we can swap one for the other. 

Lets understand with an example: 

In [ ]:
from enum import Enum

class RelationshipType(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2

class Person:
    def __init__(self, name):
        self.name = name
    

#a low level module that stores all the relationships
class Relationship:
    def __init__(self):
        self.relations = []
    
    def add_parent_child(self, parent, child):
        self.relations.append((parent, child, RelationshipType.PARENT))
        self.relations.append((child, parent, RelationshipType.CHILD))


#now in order to break the DIP, we are going to first define a high level module that should not really care about the how the relationships are stored
class Research:
    def __init__(self, relationship):
        relations = relationship.relations
        for r in relations:
            if r[0].name == "John" and r[2] == RelationshipType.PARENT:
                print(f"{r[0].name} is a parent of {r[1].name}")

parent = Person("John")
child1 = Person("Jane")
child2 = Person("Doe")

relationship = Relationship()
relationship.add_parent_child(parent, child1)
relationship.add_parent_child(parent, child2)

research = Research(relationship)  # This will print "John is a parent of Jane" and "John is a parent of Doe"
print("Research completed.")


John is a parent of Jane
John is a parent of Doe
Research completed.


All the implementation seems ok, but there is a problem with the code, it is tightly coupled with the low level module <br>
So if we want to change the way we store relationships, we will have to change the high level module as well.<br>
This will happend because in this case the high lebvel module is accesing the internal storage mechanism of the low level module.<br>


So how to tackel this situation? 

First we could define an interface for the low level module 

Remember the idea is that Research should not depend on the concrete implementation of the Relationships but it should depend on some sort of abstraction that can subsequently change 

In [ ]:
from enum import Enum
from abc import abstractmethod

class RelationshipType(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2

class Person:
    def __init__(self, name):
        self.name = name
    
#new class 
class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name):
        pass

#a low level module that stores all the relationships
class Relationship(RelationshipBrowser):  #low level module that implements the RelationshipBrowser interface
    def __init__(self):
        self.relations = []
    
    def add_parent_child(self, parent, child):
        self.relations.append((parent, child, RelationshipType.PARENT))
        self.relations.append((child, parent, RelationshipType.CHILD))
    
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[2] == RelationshipType.PARENT:
                print(f"{r[0].name} is a parent of {r[1].name}")            #new implementation of the method that is now in the low level module, given in the interface RelationshipBrowser


#now in order to break the DIP, we are going to first define a high level module that should not really care about the how the relationships are stored
class Research:  #high level module     
    # def __init__(self, relationship):
    #     relations = relationship.relations
        # for r in relations:
        #     if r[0].name == "John" and r[2] == RelationshipType.PARENT:
        #         print(f"{r[0].name} is a parent of {r[1].name}")              #this whole thing should be done in the low level module i.e. Relationship
    def __init__(self, browser: RelationshipBrowser):
        browser.find_all_children_of("John")

parent = Person("John")
child1 = Person("Jane")
child2 = Person("Doe")

relationship = Relationship()
relationship.add_parent_child(parent, child1)
relationship.add_parent_child(parent, child2)

research = Research(relationship)  # This will print "John is a parent of Jane" and "John is a parent of Doe"
print("Research completed.")


John is a parent of Jane
John is a parent of Doe
Research completed.
